In [1]:
# Python Dependencies
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


In [23]:
# read in and clean Constituent Data
constituent_data = pd.read_csv('constituents_sh.csv')


constituent_data.rename(columns={
    'Constituent type': 'constituent_type',
    'First gift amount': 'first_gift_amount',
    'Constituent codes': 'constituent_codes',
    'Lifetime giving': 'lifetime_giving',
    'First gift type': 'first_gift_type',
    'First gift date': 'first_gift_date',
    'First gift fund': 'first_gift_fund',
    'Latest gift amount': 'latest_gift_amount',
    'Latest gift date': 'latest_gift_date',
    'Latest gift fund': 'latest_gift_fund',
    'Greatest gift amount': 'greatest_gift_amount',
    'Greatest gift type': 'greatest_gift_type',
    'Greatest gift date': 'greatest_gift_date',
    'Greatest gift fund': 'greatest_gift_fund',
    'Marital status': 'marital_status',
    'Solicit codes': 'solicit_codes',
    'Student Organizations': 'student_organizations',
    'Fraternal Organizations': 'fraternal_organizations',
    "Men's Athletics": 'mens_athletics',
    "Scholarship Recipient": 'scholarship_recipient',
    "Women's Athletics": 'womens_athletics'
}, inplace=True)

# Remove rows where 'first_gift_date' is null
constituent_data_cleaned = constituent_data.dropna(subset=['first_gift_date'])

constituent_data_cleaned.head()

,ConstituentID,City,State,Postcode,Country,constituent_type,constituent_codes,lifetime_giving,first_gift_amount,first_gift_type,...,Education,Gender,marital_status,solicit_codes,student_organizations,Arts,fraternal_organizations,mens_athletics,scholarship_recipient,womens_athletics
1,43735,Hermantown,MN,55811-1755,United States,Organization,Other Organizations (8/29/2012 - No end date),50,50,One-time gift,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,60145,Esko,MN,55733-9645,United States,Organization,Other Organizations (No start date - No end date),50,50,One-time gift,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21332,NaN,NaN,NaN,United States,Organization,Business (No start date - No end date),25,25,One-time gift,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,21351,NaN,NaN,NaN,United States,Organization,Business (No start date - No end date),125,125,One-time gift,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,36583,Superior,WI,54880-2934,United States,Organization,Business (No start date - No end date),450,50,One-time gift,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# read in and clean Gift Data
gift_data = pd.read_csv('gifts_sh.csv')

gift_data.rename(columns={
    'Gift ID': 'gift_ID',
    'Is anonymous': 'is_anonymous',
    'Gift subtype': 'gift_subtype',
    'Gift constituency': 'gift_constituency',
    'Payment method': 'payment_method',
    'Online Gift': 'online_gift',
    "Superior Fund": 'Superior_fund'
}, inplace=True)

# Remove rows where 'Type' is pledge
gift_data_cleaned = gift_data[gift_data['Type'] != 'Pledge']

# Export the cleaned DataFrame to a CSV file
gift_data_cleaned.to_csv('cleaned_gift_data.csv', index=False)


C:\Users\sohof\AppData\Local\Temp\ipykernel_27560\1378609545.py:2: DtypeWarning: Columns (9,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  gift_data = pd.read_csv('gifts_sh.csv')


In [39]:
# merge data
combined_data = pd.merge(constituent_data_cleaned, gift_data_cleaned, on='ConstituentID', how='inner')